In [8]:
import os
import glob
import shutil
import random
import json
import yaml
import numpy as np

### 제작 데이터 전처리

In [4]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = float(((bbox[2] + bbox[0]) / 2) / w)
    y_center = float(((bbox[3] + bbox[1]) / 2) / h)
    width = float((bbox[2] - bbox[0]) / w)
    height = float((bbox[3] - bbox[1]) / h)
    return [x_center, y_center, width, height]

In [2]:
root_folder = '/home/pepsi/Downloads/labelme/'
change_folder = '/home/pepsi/Downloads/labelme/labels'

if not os.path.exists(change_folder):
      os.makedirs(change_folder)

file_list = glob.glob(root_folder + "*.json")
for file in file_list:
  fiie = file.split('/')[-1]
  shutil.move(os.path.join(root_folder, file), os.path.join(change_folder, fiie))

In [3]:
root_folder = '/home/pepsi/Downloads/labelme/'
change_folder = '/home/pepsi/Downloads/labelme/images'

if not os.path.exists(change_folder):
      os.makedirs(change_folder)

file_list = glob.glob(root_folder + "*.png")
file_list
for file in file_list:
  fiie = file.split('/')[-1]
  shutil.move(os.path.join(root_folder, file), os.path.join(change_folder, fiie))

In [9]:
import os
import json

label_folder = '/home/pepsi/Downloads/labelme/labels/'
new_folder = '/home/pepsi/Downloads/labelme/new_labels/'

if not os.path.isdir(new_folder):
    os.mkdir(new_folder)

classes = ['solid', 'center', 'dotted', 'stopline', 'crosswalk', 'pinky', 
           'speedlimit_30', 'speedlimit_60', 'redlight', 'yellowlight', 
           'greenlight', 'pedesrian']

label_list = os.listdir(label_folder)

for file in label_list:
    with open(os.path.join(label_folder, file), 'r') as json_file:
        lab = json.load(json_file)
        w = int(lab['imageWidth'])
        h = int(lab['imageHeight'])
        f = open(os.path.join(new_folder, file.replace('.json', '.txt')), 'w')

        for shape in lab['shapes']:
            label = shape['label']
            if label.isdigit() and int(label) < len(classes):
                idx = int(label)
            else:
                continue

            points = shape['points']
            segmentation = []

            for x, y in points:
                x_norm = x / w
                y_norm = y / h
                segmentation.extend([round(x_norm, 6), round(y_norm, 6)])

            # YOLO Segmentation format: class_id x1 y1 x2 y2 ... xn yn
            result = f"{idx} " + " ".join(map(str, segmentation)) + "\n"
            f.write(result)
        f.close()

In [10]:
if os.path.isdir('/home/pepsi/Downloads/labelme/train'):
    shutil.rmtree('/home/pepsi/Downloads/labelme/train')
if os.path.isdir('/home/pepsi/Downloads/labelme/val'):    
    shutil.rmtree('/home/pepsi/Downloads/labelme/val')

folder_list = ['/home/pepsi/Downloads/labelme/train', '/home/pepsi/Downloads/labelme/val', '/home/pepsi/Downloads/labelme/train/images', '/home/pepsi/Downloads/labelme/train/labels', '/home/pepsi/Downloads/labelme/val/images', '/home/pepsi/Downloads/labelme/val/labels']
for folder in folder_list:
    if not os.path.isdir(folder):
        os.mkdir(folder)

In [11]:
random.seed(2020)

root_folder = "/home/pepsi/Downloads/labelme"
file_folder = '/home/pepsi/Downloads/labelme/new_labels'

file_list = os.listdir(file_folder)
random.shuffle(file_list)
test_ratio = 0.1

val_list = file_list[:int(len(file_list)*test_ratio)]
train_list = file_list[int(len(file_list)*test_ratio):]

for i in val_list:
    from_file_path = os.path.join(root_folder, 'images', i[:-4]+'.png')
    to_file_path =  os.path.join(root_folder, 'val/images/',i[:-4]+'.png')
    shutil.copyfile(from_file_path, to_file_path)
    from_file_path = os.path.join(root_folder, file_folder, i)
    to_file_path =  os.path.join(root_folder, 'val/labels/',i)
    shutil.copyfile(from_file_path, to_file_path)

for i in train_list:
    from_file_path = os.path.join(root_folder, 'images', i[:-4]+'.png')
    to_file_path =  os.path.join(root_folder, 'train/images/',i[:-4]+'.png')
    shutil.copyfile(from_file_path, to_file_path)
    from_file_path = os.path.join(root_folder, file_folder, i)
    to_file_path =  os.path.join(root_folder, 'train/labels/',i)
    shutil.copyfile(from_file_path, to_file_path)

In [12]:
data =dict()

data['train'] = '/home/pepsi/Downloads/labelme/train'
data['val'] = '/home/pepsi/Downloads/labelme/val'
data['test'] = ''

data['nc'] = 12
data['names'] =['solid','center', 'dotted', 'stopline', 'crosswlaker', 'pinky', 'speedlimit_30', 'speedlimit_60', 'redlight', 'yellowlight', 'greenlight', 'pedesrian']

with open('/home/pepsi/Downloads/labelme/data.yaml', 'w') as f:
	yaml.dump(data, f)